In [1]:
import cvxpy as cp
import pulp as pl
import re
import os
import json

## Volume of a box

In [2]:
# Problem data.
A_wall = 100
A_flr = 10
alpha = 0.5
beta = 2
gamma = 0.5
delta = 2

h = cp.Variable(pos=True, name="h")
w = cp.Variable(pos=True, name="w")
d = cp.Variable(pos=True, name="d")

volume = h * w * d
wall_area = 2 * (h * w + h * d)
flr_area = w * d
hw_ratio = h/w
dw_ratio = d/w
constraints = [
    wall_area <= A_wall,
    flr_area <= A_flr,
    hw_ratio >= alpha,
    hw_ratio <= beta,
    dw_ratio >= gamma,
    dw_ratio <= delta
]
problem = cp.Problem(cp.Maximize(volume), constraints)
print(problem)

assert not problem.is_dcp()
assert problem.is_dgp()
problem.solve(gp=True)

display(problem.value)

display(h.value)
display(w.value)
display(d.value)

# A 1% increase in allowed wall space should yield approximately
# a 0.83% increase in maximum value.
display(constraints[0].dual_value)

# A 1% increase in allowed wall space should yield approximately
# a 0.66% increase in maximum value.
display(constraints[1].dual_value)

maximize h @ w @ d
subject to 2.0 @ (h @ w + h @ d) <= 100.0
           w @ d <= 10.0
           0.5 <= h / w
           h / w <= 2.0
           0.5 <= d / w
           d / w <= 2.0


np.float64(77.4596656341406)

np.float64(7.7459666205039595)

np.float64(3.8729833714161064)

np.float64(2.581988861635852)

0.8333586514592337

0.6666413498862451

## Euphemia Mixed-Integer Quadratic Program

In [3]:
def normalize_data_dict(data_dict):
    """
    Costruisce un array di oggetti 'new_data'. L'oridnamento dei periodi segue l'indicizzazione di 'new_data'. 
    """
    new_data = [] 
    for pt, data_pt in data_dict.items():
        mtu = int(pt.lower().replace("pt", ""))
        mtu_mult_factor = int(mtu/15)
        for datum in data_pt:
            p_start = (datum['period']-1)*mtu_mult_factor+1
            p_end = p_start+mtu_mult_factor
            i = 1
            for period in range(p_start, p_end):
                is_already_here = False
                if len(new_data) > 0:
                    for d_ in new_data:
                        if d_['period'] == period:
                            new_datum = d_
                            is_already_here = True
                            break
                        else:
                            new_datum = {'period': period}       
                else:
                    new_datum = {'period': period}

                if not 'OFFID' in new_datum.keys():
                    new_datum['OFFID'] = [f'{x}_{pt.lower()}' for x in datum['OFFID']]
                    new_datum['OFFList'] = datum['OFFList']
                    new_datum['OFFPrices'] = datum['OFFPrices']
                    new_datum['OFFZones'] = datum['OFFZones']
                    #new_datum['OFFMAR'] = datum['OFFMAR']
                else:
                    new_datum['OFFID'] += [f'{x}_{pt.lower()}' for x in datum['OFFID']]
                    new_datum['OFFList'] += datum['OFFList']
                    new_datum['OFFPrices'] += datum['OFFPrices']
                    new_datum['OFFZones'] += datum['OFFZones']
                    #new_datum['OFFMAR'] += datum['OFFMAR']

                if not 'BIDID' in new_datum.keys():
                    new_datum['BIDID'] = [f'{x}_{pt.lower()}' for x in datum['BIDID']]
                    new_datum['BIDList'] = datum['BIDList']
                    new_datum['BIDPrices'] = datum['BIDPrices']
                    new_datum['BIDZones'] = datum['BIDZones']
                    #new_datum['BIDMAR'] = datum['BIDMAR']
                else:
                    new_datum['BIDID'] += [f'{x}_{pt.lower()}' for x in datum['BIDID']]
                    new_datum['BIDList'] += datum['BIDList']
                    new_datum['BIDPrices'] += datum['BIDPrices']
                    new_datum['BIDZones'] += datum['BIDZones']
                    #new_datum['BIDMAR'] += datum['BIDMAR']

                if not 'Zones' in new_datum.keys():
                    new_datum['Zones'] = datum['Zones']
                    new_datum['ZoneNames'] = datum['ZoneNames']
                else:
                    #new_datum['Zones'] += datum['Zones']
                    new_datum['Zones'] = list(set(new_datum['Zones']+datum['Zones']))
                    #new_datum['ZoneNames'] += datum['ZoneNames']
                    new_datum['ZoneNames'] = list(set(new_datum['ZoneNames']+datum['ZoneNames']))

                if not is_already_here:
                    new_data.append(new_datum)
                i += 1
    return new_data

In [4]:
input_path = os.path.join('..', 'inputs')
#in_file_name = 'fully_accepted_1.json'
#in_file_name = 'fully_accepted_2.json'
in_file_name = 'partially_accepted_3.json'
#in_file_name = 'partially_accepted_4.json'
#in_file_name = 'paradoxally_rejected_5.json'

data_dict = {}
with open(os.path.join(input_path, in_file_name), 'r') as f:
    data_dict = json.load(f)

data_pt15 = data_dict["data_pt15"] if "data_pt15" in data_dict.keys() else []
data_pt30 = data_dict["data_pt30"] if "data_pt30" in data_dict.keys() else []
data_pt60 = data_dict["data_pt60"] if "data_pt60" in data_dict.keys() else []

data = normalize_data_dict({"pt15": data_pt15, "pt30": data_pt30, "pt60": data_pt60})

In [41]:
def build_miqp_problem(data, relaxed=False):
    m_rgx_ptn='^(\w+)_pt(\d{2})$'

    sub_probs = []
    constraints = []
    constraints_dict = {}
    objective = None

    g_ovs = []
    g_obvs = []    
    g_bvs = []
    g_bbvs = []

    for datum in data:
        pt15_p = datum['period']

        off_ids = datum['OFFID']
        off_lists = datum['OFFList']
        off_prices = datum['OFFPrices']
        off_zones = datum['OFFZones']
        bid_ids = datum['BIDID']
        bid_lists = datum['BIDList']
        bid_prices = datum['BIDPrices']
        bid_zones = datum['BIDZones']
        zone_ids = datum['Zones']

        ovs = []    # Vars for offers
        obvs = []   # Acceptance binary vars for offers 
        for j in range(0, len(off_ids)):
            if not re.match(m_rgx_ptn, off_ids[j]):
                continue

            vname = re.search(m_rgx_ptn, off_ids[j]).group(1)
            mtu = int(re.search(m_rgx_ptn, off_ids[j]).group(2))
            if mtu == 15:
                p = pt15_p
            else:
                mtu_mult_factor = int(mtu/15)
                p = int((pt15_p-1)/mtu_mult_factor)+1
            
            obv = next((v for v in g_obvs if v.name() == f"off_acceptance_{vname}_pt{mtu}_{p}"), None)
            if obv is None:
                obv = cp.Variable(name=f"off_acceptance_{vname}_pt{mtu}_{p}", pos=True, bounds=[0, 1], integer=True if not relaxed else False)
                g_obvs.append(obv)
            obvs.append(obv)

            ov = next((v for v in g_ovs if v.name() == f"off_{vname}_pt{mtu}_{p}"), None)
            if ov is None:
                ov = cp.Variable(name=f"off_{vname}_pt{mtu}_{p}", pos=True, bounds=[0, off_lists[j]], integer=False)
                g_ovs.append(ov)
            ovs.append(ov)

        bvs = []    # Vars for offers
        bbvs = []   # Acceptance binary vars for bids 
        for j in range(0, len(bid_ids)):
            if not re.match(m_rgx_ptn, bid_ids[j]):
                continue
            
            vname = re.search(m_rgx_ptn, bid_ids[j]).group(1)
            mtu = int(re.search(m_rgx_ptn, bid_ids[j]).group(2))
            if mtu == 15:
                p = pt15_p
            else:
                mtu_mult_factor = int(mtu/15)
                p = int((pt15_p-1)/mtu_mult_factor)+1

            bbv = next((v for v in g_bbvs if v.name() == f"bid_acceptance_{vname}_pt{mtu}_{p}"), None)
            if bbv is None:
                bbv = cp.Variable(name=f"bid_acceptance_{vname}_pt{mtu}_{p}", pos=True, bounds=[0, 1], integer=True if not relaxed else False)
                g_bbvs.append(bbv)
            bbvs.append(bbv)
            
            bv = next((v for v in g_bvs if v.name() == f"bid_{vname}_pt{mtu}_{p}"), None)
            if bv is None:
                bv = cp.Variable(name=f"bid_{vname}_pt{mtu}_{p}", pos=True, bounds=[0, bid_lists[j]], integer=False)
                g_bvs.append(bv)
            bvs.append(bv)

        #sub_probs.append(cp.sum([bbvs[j]*bvs[j]*bid_prices[j] for j in range(0, len(bid_lists))]) - cp.sum([obvs[j]*ovs[j]*off_prices[j] for j in range(0, len(off_lists))]))
        sub_probs.append(cp.sum([bvs[j]*bid_prices[j] for j in range(0, len(bid_lists))]) - cp.sum([ovs[j]*off_prices[j] for j in range(0, len(off_lists))]))

        # Constarint 1
        constraint_1_name = f"balance_bids_offs_{pt15_p}"        
        #constraint_1 = cp.Zero(cp.sum( [bbvs[j]*bvs[j] for j in range(0, len(bvs))] ) - cp.sum( [obvs[j]*ovs[j] for j in range(0, len(ovs))] ))
        constraint_1 = cp.Zero(cp.sum( bvs ) - cp.sum( ovs ))        
        if constraint_1_name not in constraints_dict.keys():
            constraints.append(constraint_1)
            constraints_dict[constraint_1_name] = constraint_1.constr_id

        for z in zone_ids:
            # Constraint 2
            constraint_2_name = f"zone_balance_{z}_{pt15_p}"
            #constraint_2 = cp.Zero(cp.sum([ bbvs[i]*v for i,v in enumerate(bvs) if bid_zones[i] == z ]) - cp.sum([ obvs[i]*v for i,v in enumerate(ovs) if off_zones[i] == z ]))
            constraint_2 = cp.Zero(cp.sum([ v for i,v in enumerate(bvs) if bid_zones[i] == z ]) - cp.sum([ v for i,v in enumerate(ovs) if off_zones[i] == z ]))
            if constraint_2 not in constraints_dict.keys():
                constraints.append(constraint_2)
                constraints_dict[constraint_2_name] = constraint_2.constr_id
            
    # Objective function to maximize
    objective = cp.Maximize(cp.sum(sub_probs))

    prob = cp.Problem(objective, constraints)
    
    return prob, constraints_dict

In [42]:
problem_name = f'euphemia_miqp_{in_file_name.replace(".json", "")}'
relaxed = False

prob, constraints_dict = build_miqp_problem(data, relaxed=relaxed)
constraints_dict_rev = dict((v,k) for k,v in constraints_dict.items())

print(prob)

#assert not prob.is_dcp()
#assert prob.is_dgp()
#prob.solve(gp=True)

prob.solve(gp=False)

display(prob.value.item())

maximize bid_a_pt60_1 @ 30.0 + -off_b_pt15_1 @ 25.0 + bid_a_pt60_1 @ 30.0 + -off_c_pt15_2 @ 26.0 + bid_a_pt60_1 @ 30.0 + -off_d_pt15_3 @ 27.0 + bid_a_pt60_1 @ 30.0 + -off_e_pt15_4 @ 28.0
subject to bid_a_pt60_1 + -off_b_pt15_1 == 0
           bid_a_pt60_1 + -off_b_pt15_1 == 0
           bid_a_pt60_1 + -off_c_pt15_2 == 0
           bid_a_pt60_1 + -off_c_pt15_2 == 0
           bid_a_pt60_1 + -off_d_pt15_3 == 0
           bid_a_pt60_1 + -off_d_pt15_3 == 0
           bid_a_pt60_1 + -off_e_pt15_4 == 0
           bid_a_pt60_1 + -off_e_pt15_4 == 0


419.99999998622764

In [38]:
m_rgx_ptn='^(\w+)_pt(\d{2})$'

ov_results = []
bv_results = []
obv_results = []
bbv_results = []
balance_zone_prices_results = []
zone_prices_results = []

for datum in data:
    pt15_p = datum['period']

    off_ids = datum['OFFID']
    bid_ids = datum['BIDID']

    for j in range(0, len(off_ids)):
        vname = re.search(m_rgx_ptn, off_ids[j]).group(1)
        mtu = int(re.search(m_rgx_ptn, off_ids[j]).group(2))
        if mtu == 15:
            p = pt15_p
        else:
            mtu_mult_factor = int(mtu/15)
            p = int((pt15_p-1)/mtu_mult_factor)+1

        ov_rgx_ptn = f'^off_({vname})_pt{mtu}_{p}$'
        ov_result = {re.search(ov_rgx_ptn, v.name()).group(1): round(v.value.item(),4) if v.value is not None else None for v in prob.variables() \
                    if re.match(ov_rgx_ptn, v.name(), re.IGNORECASE)}
        ov_results.append(ov_result)

        obv_rgx_ptn = f'^off_acceptance_({vname})_pt{mtu}_{p}$'
        obv_result = {re.search(obv_rgx_ptn, v.name()).group(1): round(v.value.item(),4) if v.value is not None else None for v in prob.variables() \
                    if re.match(obv_rgx_ptn, v.name(), re.IGNORECASE)}
        obv_results.append(obv_result)

    for j in range(0, len(bid_ids)):
        vname = re.search(m_rgx_ptn, bid_ids[j]).group(1)
        mtu = int(re.search(m_rgx_ptn, bid_ids[j]).group(2))
        if mtu == 15:
            p = pt15_p
        else:
            mtu_mult_factor = int(mtu/15)
            p = int((pt15_p-1)/mtu_mult_factor)+1
    
        bv_rgx_ptn = f'^bid_({vname})_pt{mtu}_{p}$'
        bv_result = {re.search(bv_rgx_ptn, v.name()).group(1): round(v.value.item(),4) if v.value is not None else None for v in prob.variables() \
                    if re.match(bv_rgx_ptn, v.name(), re.IGNORECASE)}
        bv_results.append(bv_result)

        bbv_rgx_ptn = f'^bid_acceptance_({vname})_pt{mtu}_{p}$'
        bbv_result = {re.search(bbv_rgx_ptn, v.name()).group(1): round(v.value.item(),4) if v.value is not None else None for v in prob.variables() \
                    if re.match(bbv_rgx_ptn, v.name(), re.IGNORECASE)}
        bbv_results.append(bbv_result)

    # Balance Bids/Offs in dual mode (price?)        
    balance_main = next(c.dual_value for c in prob.constraints if c.constr_id == constraints_dict[f"balance_bids_offs_{pt15_p}"])
    # Balance Zone in dual mode (shadow proces)
    zone_blanace_contraint_regex = re.compile(f'^zone_balance_(\w+)_{pt15_p}$')
    balance_zone_prices_result = { 
        int(re.search(zone_blanace_contraint_regex, constraints_dict_rev[c.constr_id]).group(1)) : balance_main - round(c.dual_value,2) 
        for c in prob.constraints if zone_blanace_contraint_regex.match(constraints_dict_rev[c.constr_id])
    }
    zone_prices_result = {
        int(re.search(zone_blanace_contraint_regex, constraints_dict_rev[c.constr_id]).group(1)) : round(c.dual_value,2) 
        for c in prob.constraints if zone_blanace_contraint_regex.match(constraints_dict_rev[c.constr_id])
    }
    balance_zone_prices_results.append(balance_zone_prices_result)
    zone_prices_results.append(zone_prices_result)

print(f"Problem Name: {problem_name} - Relaxed: {relaxed}")
print(f"Problem status: {prob.status}")
print(f"Problem objective: {round(pl.value(prob.value), 2)}")
print(f"offs: {ov_results}")
print(f"bids: {bv_results}")
print(f"offs acceptance: {obv_results}")
print(f"bids acceptance: {bbv_results}")
print(f"balance_zone_prices_results: {balance_zone_prices_results}")
print(f"zone_prices_results: {zone_prices_results}")

Problem Name: euphemia_miqp_partially_accepted_3 - Relaxed: False
Problem status: optimal
Problem objective: 420.0
offs: [{'b': 30.0}, {'c': 30.0}, {'d': 30.0}, {'e': 30.0}]
bids: [{'a': 30.0}, {'a': 30.0}, {'a': 30.0}, {'a': 30.0}]
offs acceptance: [{}, {}, {}, {}]
bids acceptance: [{}, {}, {}, {}]
balance_zone_prices_results: [{1: 1.640250822276812e-06}, {1: -1.8503063614616622e-07}, {1: -5.772955713467809e-07}, {1: -4.7361031310089174e-06}]
zone_prices_results: [{1: 12.5}, {1: 20.0}, {1: 13.5}, {1: 14.0}]


In [14]:
print(prob)

maximize bid_a_pt60_1 @ 30.0 + -off_b_pt15_1 @ 25.0 + bid_a_pt60_1 @ 30.0 + -off_c_pt15_2 @ 26.0 + bid_a_pt60_1 @ 30.0 + -off_d_pt15_3 @ 27.0 + bid_a_pt60_1 @ 30.0 + -off_e_pt15_4 @ 28.0
subject to bid_a_pt60_1 + -off_b_pt15_1 == 0
           bid_a_pt60_1 + -off_b_pt15_1 == 0
           bid_a_pt60_1 + -off_c_pt15_2 == 0
           bid_a_pt60_1 + -off_c_pt15_2 == 0
           bid_a_pt60_1 + -off_d_pt15_3 == 0
           bid_a_pt60_1 + -off_d_pt15_3 == 0
           bid_a_pt60_1 + -off_e_pt15_4 == 0
           bid_a_pt60_1 + -off_e_pt15_4 == 0


## LP vs QP Solver

In [24]:
x1 = pl.LpVariable('x1', lowBound=0)
x2 = pl.LpVariable('x2', lowBound=0)

prob = pl.LpProblem(name='LP_easy', sense=pl.LpMaximize)
prob += 130*x1 + 100*x2
prob += 1.5*x1 + x2 <= 27
prob += x1 + x2 <= 21
prob += 0.3*x1 + 0.5*x2 <= 9

prob.solve(pl.PULP_CBC_CMD(msg=False)) 

print(f"Problem status: {pl.LpStatus[prob.status]}")
print(f"Problem objective: {round(pl.value(prob.objective), 4)}")

print(f"Problem vars")
for v in prob.variables():
    print(f"{v.name}: {round(v.varValue, 4)}")

print(f"Problem duals")
for name, c in prob.constraints.items():
    print(f"{name}: {round(c.pi, 4)}")

Problem status: Optimal
Problem objective: 2460.0
Problem vars
x1: 12.0
x2: 9.0
Problem duals
_C1: 60.0
_C2: 40.0
_C3: -0.0


In [28]:
x1 = cp.Variable(name=f'x1', pos=True, integer=False)
x2 = cp.Variable(name=f'x2', pos=True, integer=False)

vars = [x1, x2]
#constraints = [
#    1.5*x1 + x2 <= 27,
#    x1 + x2 <= 21,
#    0.3*x1 + 0.5*x2 <= 9
#]
constraints = []
constraints_dict = {}
c1 = cp.NonPos(1.5*x1 + x2 - 27)
constraints.append(c1)
constraints_dict["_C1"] = c1.constr_id
c2 = cp.NonPos(x1 + x2 - 21)
constraints.append(c2)
constraints_dict["_C2"] = c2.constr_id
c3 = cp.NonPos(0.3*x1 + 0.5*x2 - 9)
constraints.append(c3)
constraints_dict["_C3"] = c3.constr_id

objective = cp.Maximize(130*x1 + 100*x2)
prob = cp.Problem(objective, constraints)

prob.solve(gp=False)

print(f"Problem status: {prob.status}")
print(f"Problem objective: {round(pl.value(prob.value), 4)}")

print(f"Problem vars")
for name, v in prob.var_dict.items():
    print(f"{name}: {round(v.value.item(), 4) if v.value is not None else None}")

print(f"Problem duals")
for name, id in constraints_dict.items():
    c = next(c for c in prob.constraints if c.constr_id == id)
    print(f"{name}: {round(c.dual_value, 4)}")


Problem status: optimal
Problem objective: 2460.0
Problem vars
x1: 12.0
x2: 9.0
Problem duals
_C1: 60.0
_C2: 40.0
_C3: 0.0
